## **CLIP model (Contrastive Language–Image Pretraining)**

The **CLIP model (Contrastive Language–Image Pretraining)**, introduced by **OpenAI (Radford et al., 2021)**, is a **multimodal neural network** that learns to connect **images and text** through **contrastive learning** — essentially teaching the model to *understand* which image matches which caption.

---

### 1. Motivation

Before CLIP, most vision models (like ResNet, ViT) were trained **supervised** — e.g., ImageNet classification with 1000 fixed labels.
But these models could not generalize to *new* concepts or tasks.

CLIP instead learns from **natural language supervision** — hundreds of millions of (image, text) pairs collected from the web — so it learns a *broad* visual-language understanding without task-specific labels.

---

### 2. Core Idea

The idea is simple but powerful:

> Learn a joint embedding space where **matching image–text pairs** have **high similarity**, and **non-matching pairs** have **low similarity**.

That is, if we encode an image and its caption, their embeddings should be close; random combinations should be far apart.

---

### 3. Architecture

CLIP consists of **two encoders** trained jointly:

| Encoder           | Example Architecture     | Input | Output       |
| ----------------- | ------------------------ | ----- | ------------ |
| **Image Encoder** | ResNet-50 / ViT-B/32     | Image | 512-D vector |
| **Text Encoder**  | Transformer (like GPT-2) | Text  | 512-D vector |

Both encoders project to the **same latent space** via learned linear projections.

---

#### 3.1. Image Encoder

For example, if using ViT-B/32:

* The image is divided into 32×32 patches.
* Each patch becomes a token.
* Tokens are passed through the Vision Transformer.
* The final [CLS] token gives a **feature vector** of dimension `D=512`.

$$
\mathbf{v} = f_{\text{img}}(I) \in \mathbb{R}^{512}
$$

---

#### 3.2. Text Encoder

The text is tokenized (e.g., with BPE), and a Transformer encodes it.
The last token’s hidden state (or a [EOS] token) gives the text representation:

$$
\mathbf{t} = f_{\text{text}}(T) \in \mathbb{R}^{512}
$$

---



Excellent and very insightful question — and you’re absolutely right:
in the **standard ResNet-50**, the final “head” is a **fully connected (FC)** layer that outputs **class logits**, e.g. 1000 classes for ImageNet.

But in **CLIP**, they *modify* the ResNet so that instead of predicting fixed labels, it produces a **feature embedding vector** (e.g., 512-D).

Let’s go step-by-step through what happens.

---

## 1. Standard ResNet-50 Recap

A normal ResNet-50 has this structure:

```
Input (224×224)
→ Conv1 + BN + ReLU + MaxPool
→ Conv2_x
→ Conv3_x
→ Conv4_x
→ Conv5_x
→ Global Average Pool (GAP)
→ Fully Connected layer (1000 classes)
```

The last two layers are:

1. **Global average pooling** over the spatial dimension → shape becomes `[B, 2048]`
2. **FC layer** → `[B, 1000]` logits

So the last layer is specific to **classification**.

---

## 2. How CLIP Uses ResNet-50

CLIP **removes the classification head** and replaces it with a **projection head** that maps features to a shared text–image embedding space.

The pipeline becomes:

$$
\text{Image} \xrightarrow{f_{\text{ResNet}}} \mathbf{h}_{\text{img}} \in \mathbb{R}^{2048} \xrightarrow{W_p} \mathbf{v} \in \mathbb{R}^{512}
$$

where:

* ( f_{\text{ResNet}} ) is the convolutional body up to the global pooling layer.
* ( W_p \in \mathbb{R}^{2048 \times 512} ) is a **learned linear projection**.
* The output ( \mathbf{v} ) is **L2-normalized**.

---

## 3. Architectural Modifications in CLIP-ResNet-50

OpenAI didn’t use the *vanilla* ResNet-50 directly — they made a few small but important changes to improve alignment with the text encoder.

| Change                                     | Description                                                                                                  | Why                                                                                    |
| ------------------------------------------ | ------------------------------------------------------------------------------------------------------------ | -------------------------------------------------------------------------------------- |
| **Replace AvgPool with Attention Pooling** | Instead of global average pooling, CLIP uses a **multi-head attention pooling** layer over spatial features. | This allows the model to **learn spatial weighting** (like a soft “CLS” token in ViT). |
| **Projection layer**                       | A new linear layer projects 2048→512                                                                         | To match text embedding dimension.                                                     |
| **L2 normalization**                       | Normalize embeddings before computing cosine similarity                                                      | Makes the contrastive loss stable.                                                     |
| **No classifier head**                     | The ImageNet classifier is removed                                                                           | CLIP doesn’t predict classes directly.                                                 |

---

## 4. In Pseudocode

Here’s a simplified PyTorch-style pseudocode:

```python
class CLIPImageEncoder(nn.Module):
    def __init__(self, backbone="resnet50", embed_dim=512):
        super().__init__()
        resnet = torchvision.models.resnet50(pretrained=False)
        modules = list(resnet.children())[:-2]  # remove avgpool + fc
        self.conv_body = nn.Sequential(*modules)
        self.attn_pool = AttentionPool2d(spatial_dim=7, embed_dim=2048, num_heads=32)
        self.proj = nn.Linear(2048, embed_dim)

    def forward(self, x):
        x = self.conv_body(x)         # [B, 2048, 7, 7]
        x = self.attn_pool(x)         # [B, 2048]
        x = self.proj(x)              # [B, 512]
        x = x / x.norm(dim=-1, keepdim=True)
        return x
```

✅ Here, the **attention pooling** layer replaces the average pooling:

$$
\text{AttentionPool}(X) = \text{softmax}\left(\frac{Q K^\top}{\sqrt{d}}\right)V
$$

It learns where to “look” spatially in the feature map when forming the final embedding.

---

## 5. Why Not Use the Classification Head?

Because CLIP’s goal is **cross-modal embedding**, not classification.

* The ResNet backbone extracts *visual semantics*.
* The projection head aligns them to the **text encoder’s semantic space**.
* During training, gradients from the **contrastive loss** update both the image and text encoders so that “a dog” and its picture get close in the shared embedding space.

---

## 6. Visualization of the CLIP-ResNet Flow

```
Image (224×224)
↓
ResNet Backbone (conv1–conv5)
↓
Feature Map (7×7×2048)
↓
Attention Pooling (learned global descriptor)
↓
Linear Projection (2048→512)
↓
L2 Normalization
↓
Image Embedding (512-D)
```

---

## 7. Key Equation Summary

$$
\begin{aligned}
\mathbf{h}*{\text{img}} &= f*{\text{ResNet}}(I) \
\mathbf{v} &= \frac{W_p \mathbf{h}*{\text{img}}}{| W_p \mathbf{h}*{\text{img}} |}
\end{aligned}
$$

The embedding ( \mathbf{v} ) is then used in the **contrastive similarity** computation with text embeddings ( \mathbf{t} ).

---

Would you like me to also show how the **attention pooling** at the end of CLIP’s ResNet-50 actually works (mathematically and in PyTorch)? It’s one of the most elegant differences compared to the vanilla ResNet.



### 4. Contrastive Training Objective

The training set contains batches of **N image–text pairs**.

For each pair ( (I_i, T_i) ):

1. Encode them into embeddings ( \mathbf{v}_i ) and ( \mathbf{t}_i ).
2. Normalize them to unit length.
3. Compute a similarity matrix ( S \in \mathbb{R}^{N \times N} ):

$$
S_{ij} = \tau \cdot (\mathbf{v}_i^\top \mathbf{t}_j)
$$

where ( \tau ) is a learnable temperature parameter.

4. The model is trained using **symmetric cross-entropy loss**:

   * For each image, the correct caption should be most similar.
   * For each caption, the correct image should be most similar.

$$
\mathcal{L}*{\text{image}} = -\frac{1}{N} \sum_i \log \frac{\exp(S*{ii})}{\sum_j \exp(S_{ij})}
$$

$$
\mathcal{L}*{\text{text}} = -\frac{1}{N} \sum_i \log \frac{\exp(S*{ii})}{\sum_j \exp(S_{ji})}
$$

$$
\mathcal{L} = \frac{1}{2}(\mathcal{L}*{\text{image}} + \mathcal{L}*{\text{text}})
$$

✅ This encourages **matching pairs** (diagonal of S) to have high cosine similarity.

---

## 5. Zero-Shot Inference

Once trained, CLIP can perform **zero-shot classification** without explicit retraining:

1. Define class names (e.g., “cat”, “dog”, “car”).
2. Convert them to prompts:
   *“a photo of a cat”*, *“a photo of a dog”*, …
3. Encode all prompts with the **text encoder**.
4. Encode the image with the **image encoder**.
5. Compute cosine similarity between the image embedding and all text embeddings.
6. Pick the most similar — that’s the predicted label.

CLIP effectively transforms **natural language descriptions into classifiers**.

---

## 6. Summary of Training and Usage

| Stage       | Data                     | Objective                 | Output                 |
| ----------- | ------------------------ | ------------------------- | ---------------------- |
| Pretraining | 400M (image, text) pairs | Contrastive loss          | Joint embedding space  |
| Inference   | Any image/text           | Similarity-based matching | Zero-shot or retrieval |

---

## 7. Applications

* **Zero-shot image classification**
* **Image–text retrieval**
* **Text-guided image generation** (used in **DALL·E**, **Stable Diffusion**, etc.)
* **Vision–language understanding** foundation (used in **BLIP**, **CLIPSeg**, **ALIGN**, etc.)

---

## 8. Intuition

* Instead of *predicting a label*, CLIP learns *which text describes this image best*.
* Language provides a **rich supervision signal** — it covers semantics much broader than fixed class labels.
* Because it learns a joint embedding space, it generalizes across **modalities**, **tasks**, and **domains**.

---



## **Numerical Example**

Let’s walk through a **toy numerical example** that illustrates exactly how CLIP’s **contrastive loss** works. 

We’ll use very small vectors and no neural networks — just random embeddings — to show how the **similarity matrix** and **loss** are computed.

---

### 1. Setup

Assume a **batch of N = 3 (image, text)** pairs:

| Pair | Image      | Text    |
| ---- | ---------- | ------- |
| 1    | 🖼 “a cat” | “a cat” |
| 2    | 🖼 “a dog” | “a dog” |
| 3    | 🖼 “a car” | “a car” |

---

### 2. Encoded Embeddings (already normalized)

Let’s say our encoders output 2-dimensional **unit vectors** (after normalization):

$$
v_1 = [0.9, 0.1], \quad t_1 = [0.8, 0.2] \
v_2 = [0.1, 0.9], \quad t_2 = [0.2, 0.8] \
v_3 = [0.9, -0.1], \quad t_3 = [1.0, 0.0]
$$

---

### 3. Compute Similarity Matrix ( S = v_i^\top t_j )

Each entry ( S_{ij} ) is the **dot product** between image ( i ) and text ( j ):

| Image\Text | t₁                           | t₂                           | t₃                           |
| ---------- | ---------------------------- | ---------------------------- | ---------------------------- |
| **v₁**     | 0.9×0.8 + 0.1×0.2 = **0.74** | 0.9×0.2 + 0.1×0.8 = 0.26     | 0.9×1.0 + 0.1×0 = 0.9        |
| **v₂**     | 0.1×0.8 + 0.9×0.2 = 0.26     | 0.1×0.2 + 0.9×0.8 = **0.74** | 0.1×1.0 + 0.9×0 = 0.1        |
| **v₃**     | 0.9×0.8 + (−0.1)×0.2 = 0.70  | 0.9×0.2 + (−0.1)×0.8 = 0.10  | 0.9×1.0 + (−0.1)×0 = **0.9** |

So:

$$
S =
\begin{bmatrix}
0.74 & 0.26 & 0.9 \
0.26 & 0.74 & 0.1 \
0.70 & 0.10 & 0.9
\end{bmatrix}
$$

---

### 4. Add Temperature Scaling

CLIP uses a learnable **temperature parameter** ( \tau ) (often ≈ 1/0.07).
Let’s just take ( \tau = 1 ) for simplicity.

---

### 5. Compute Image→Text Probabilities

For each **image** ( i ), we apply **softmax** over the row ( S_{i,:} ):

Example for image 1:

$$
p(t_j|v_1) = \frac{\exp(S_{1j})}{\sum_k \exp(S_{1k})}
$$

Compute:

* exp(0.74) = 2.10
* exp(0.26) = 1.30
* exp(0.90) = 2.46
  Sum = 5.86

So probabilities:

* p(t₁|v₁) = 2.10 / 5.86 = 0.36
* p(t₂|v₁) = 1.30 / 5.86 = 0.22
* p(t₃|v₁) = 2.46 / 5.86 = 0.42

Correct match is **t₁**, so its log-prob = log(0.36) = −1.02.

---

### 6. Repeat for other rows

| Image | exp(scores)        | Sum  | Correct prob | −log(prob) |
| ----- | ------------------ | ---- | ------------ | ---------- |
| v₁    | [2.10, 1.30, 2.46] | 5.86 | 0.36         | **1.02**   |
| v₂    | [1.30, 2.10, 1.10] | 4.50 | 0.47         | **0.75**   |
| v₃    | [2.01, 1.11, 2.46] | 5.58 | 0.36         | **1.02**   |

Average image→text loss:

$$
\mathcal{L}_{\text{image}} = \frac{1}{3}(1.02 + 0.75 + 1.02) = 0.93
$$

---

## 7. Text→Image Loss (symmetric)

Do the same for **columns** (softmax over Sᵀ).
You’ll get a similar value (≈0.93).

Final CLIP loss:

$$
\mathcal{L} = \frac{1}{2}(\mathcal{L}*{\text{image}} + \mathcal{L}*{\text{text}}) \approx 0.93
$$

---

## 8. Intuition

✅ When matching pairs (diagonal) are **larger** than off-diagonal ones,
the model gets **low loss** (better alignment).

❌ If the similarity matrix has mixed values (e.g., an image looks similar to a wrong text), the loss increases — forcing the encoders to *separate* them more.

---

## 9. Conceptual Visualization

The embeddings are pushed such that:

* **Positive pairs**: ( v_i ) and ( t_i ) are close.
* **Negative pairs**: ( v_i ) and ( t_j ) (for ( j \neq i )) are far apart.

It’s a **bidirectional contrastive learning**.

---


### CLIP Training set

Excellent question — and it goes straight to the **core strength** (and a bit of the *mystery*) behind CLIP’s power.

Let’s unpack this precisely.

---

## 1. Source of CLIP’s Training Data

CLIP was trained on a dataset called **“WebImageText” (WIT)** —
a **private dataset** created by **OpenAI** that contains **≈400 million (image, text)** pairs.

This dataset was **collected from the public internet** — specifically from sources where **images naturally co-occur with descriptive text**.

Typical sources include:

| Type                         | Example                                             |
| ---------------------------- | --------------------------------------------------- |
| **Alt-text from web images** | HTML `<img>` tags with `alt=` descriptions          |
| **Image captions**           | Stock photo sites, image-sharing sites, articles    |
| **Social media posts**       | Tweets, Reddit, etc. containing both image and text |
| **Public web pages**         | Where images appear next to relevant sentences      |
| **Metadata & filenames**     | e.g., `cat_on_a_bed.jpg` or `dog_playing_ball.png`  |

The OpenAI team used a **web crawler** to gather image URLs and associated surrounding text, then applied **filtering and deduplication** to form the final dataset.

---

## 2. Scale and Diversity

* **400 million pairs**
* **32,768 GPUs** (distributed across clusters)
* Data from **hundreds of thousands of websites**
* Roughly **5 billion text tokens**
* Huge diversity: objects, scenes, actions, memes, artworks, etc.

That diversity gives CLIP its **broad generalization** — it learns what a “photo of a cat” means without ever seeing the word “cat” as a *class label*.

---

## 3. Data Cleaning and Filtering

Raw internet data is noisy — so OpenAI applied several steps:

1. **Text filtering**

   * Remove short, meaningless strings (e.g., “IMG_001.jpg”)
   * Exclude non-English or overly rare languages
   * Basic profanity and NSFW filtering

2. **Image validation**

   * Check that URLs actually lead to valid images
   * Ensure diversity (avoid duplicates or corrupted images)

3. **Pair quality filtering**

   * Compute similarity between preliminary text and image embeddings
   * Discard pairs that clearly mismatch (e.g., cat image + random quote)

---

## 4. Comparison to Other Datasets

| Dataset        | Size | Type                              | Public?   |
| -------------- | ---- | --------------------------------- | --------- |
| **OpenAI WIT** | 400M | Web image–text pairs              | ❌ Private |
| **LAION-400M** | 400M | Web crawl based on CLIP filtering | ✅ Public  |
| **LAION-5B**   | 5B   | Massive extension, multilingual   | ✅ Public  |
| **CC3M**       | 3M   | Curated “Conceptual Captions”     | ✅ Public  |
| **CC12M**      | 12M  | Larger CC version                 | ✅ Public  |

So, when OpenAI released the CLIP paper, they didn’t release the original **WIT-400M** dataset.
But later, **LAION** (a community effort) **recreated** it by using the *same strategy*:
use a web crawler, grab image–text pairs, and filter them using CLIP’s own similarity score.

---

## 5. Summary

| Aspect              | Description                                             |
| ------------------- | ------------------------------------------------------- |
| Dataset             | WebImageText (WIT)                                      |
| Size                | ~400M image–text pairs                                  |
| Source              | Public internet (HTML, captions, metadata)              |
| Publicly released?  | ❌ No (OpenAI internal)                                  |
| Public alternatives | ✅ LAION-400M, LAION-5B                                  |
| Purpose             | Learn cross-modal alignment between vision and language |

---

## 6. Key Insight

CLIP didn’t need manually labeled data like “ImageNet-dog-class.”
Instead, it learned *from how humans naturally describe images online.*

That’s why it can recognize **open-vocabulary** concepts — things it’s never seen in labeled datasets — simply by matching the semantics of text and vision.

---

Would you like me to show how **LAION-400M** (the public version) can be **downloaded and filtered** using the CLIP model for your own experiments (e.g., building your own CLIP-style dataset)?
